In [207]:
import pandas as pd
import numpy as np
import biom
import re
from biom.util import biom_open  
from biom.table import Table
import json
import os

In [208]:
def read_csv(meta_fp, biom_fp, min_freq_sum, min_freq):
    meta = pd.read_csv(meta_fp, sep='\t', dtype={'#SampleID': str}, low_memory=False, index_col= '#SampleID')
    table = biom.load_table(biom_fp)
    print(meta_fp, meta.shape, biom_fp, table.shape)
    meta = meta.loc[meta.index.isin(table.ids('sample'))]
    table = table.filter(axis='sample',ids_to_keep=meta.index, inplace=False)
    print(meta_fp, meta.shape, biom_fp, table.shape)
    filter_fn = lambda val, id_, md: val.sum() > min_freq_sum
    table = table.filter(axis='observation', ids_to_keep=filter_fn, inplace=False)
    filter_fn = lambda val, id_, md: val.sum() > min_freq  
    print(meta_fp, meta.shape, biom_fp, table.shape)
    table = table.filter(axis='sample', ids_to_keep=filter_fn, inplace=False)
    print(meta_fp, meta.shape, biom_fp, table.shape)
    meta = meta.loc[meta.index.isin(table.ids('sample'))]
    return meta, table

In [209]:
main_dir = '/Users/erfan/Main/oasis/test_augmentation/augmentation/AGP/'
biom_fp = main_dir + '/all.biom'
meta_fp = main_dir + '/20180405_cleaned_agp_metadata.tsv'

In [212]:
metat1, tablet1 = read_csv(meta_fp, biom_fp, 1, 999)

/Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//metadata.csv (17876, 537) /Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//all.biom (848984, 14794)
/Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//metadata.csv (13977, 537) /Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//all.biom (848984, 13977)
/Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//metadata.csv (13977, 537) /Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//all.biom (587689, 13977)
/Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//metadata.csv (13977, 537) /Users/erfan/Main/oasis/test_augmentation/augmentation/AGP//all.biom (587689, 13859)


In [213]:
metat1 = metat1.loc[metat1.index.isin(tablet1.ids('sample'))]

In [214]:
def filter_biom(meta, table, min_freq_sum, min_freq):
    table = table.filter(axis='sample', ids_to_keep=meta.index,inplace=False)
    filter_fn = lambda val, id_, md: val.sum() >= min_freq_sum
    table = table.filter(axis='observation', ids_to_keep=filter_fn, inplace=False)
    filter_fn = lambda val, id_, md: val.sum() >= min_freq  
    print("meta_fp", meta.shape, "biom_fp", table.shape)
    table = table.filter(axis='sample', ids_to_keep=filter_fn, inplace=False)
    print("meta_fp", meta.shape, "biom_fp", table.shape)
    meta = meta.loc[meta.index.isin(table.ids('sample'))]
    meta = meta.reindex(table.ids('sample'))
    print((meta.index == table.ids('sample')).sum())
    return meta, table

In [217]:
def filter_samples_AGP(meta, table):
    print(meta.shape, table.shape)
    to_remove_disease = ['Diagnosed by an alternative medicine practitioner', 'Diagnosed by a medical professional (doctor, physician assistant)',
                        'Self-diagnosed']
    to_remove_disease_doc = ['Diagnosed by an alternative medicine practitioner', 'Diagnosed by a medical professional (doctor, physician assistant)']
    meta = meta.loc[~meta.age_years.isin(['Unknown', 'not provided','Not provided'])]
    meta.age_years = meta.age_years.astype(float)
    meta = meta.loc[(meta.age_years >= 20) & (meta.age_years <= 70)]
    meta = meta.loc[meta.subset_healthy == True]

    meta = meta.loc[meta.antibiotic_history.isin(["I have not taken antibiotics in the past year."])]
    meta = meta.loc[meta.country.isin(['USA'])]
    meta = meta.loc[meta.diet_type.isin(['Omnivore'])]
    meta = meta.loc[meta.sample_type.isin(['Stool'])]
    meta = meta.loc[meta.bmi_cat.isin(['Normal', 'Overweight'])]
    meta = meta.loc[meta.race.isin(['Caucasian'])]
    
    meta = meta.loc[~meta.sex.isin(['Unknown','other', 'unspecified'])]

    meta = meta.loc[meta.ibd.isin(['I do not have this condition'])]

    meta = meta.loc[meta.diabetes.isin(['I do not have this condition'])]
    meta = meta.loc[~meta.autoimmune.isin(to_remove_disease_doc)]

    meta = meta.loc[~meta.cancer.isin(to_remove_disease)]
    meta = meta.loc[~meta.cardiovascular_disease.isin(to_remove_disease)]
    meta = meta.loc[~meta.kidney_disease.isin(to_remove_disease)]
    meta = meta.loc[~meta.mental_illness.isin(['true'])]
    meta = meta.loc[~meta.migraine.isin(to_remove_disease)]
    meta = meta.loc[~meta.pregnant.isin(['true','Not sure'])]
    meta = meta.loc[~meta.index.isin(['10317.000066520'])]
    print(meta.shape, table.shape)
    meta, table = filter_biom(meta, table, 4, 1000)
    return meta, table

In [218]:
meta1, table1 = filter_samples_AGP(metat1, tablet1)

(13859, 537) (587689, 13859)
(1942, 537) (587689, 13859)
meta_fp (1942, 537) biom_fp (21050, 1942)
meta_fp (1942, 537) biom_fp (21050, 1942)
1942


In [219]:
a = set(table1.ids('observation')) - set(table_main.ids('observation'))
set(table_main.ids('observation')) - set(table1.ids('observation')) 

set()

In [220]:
(table1.matrix_data - table_main.matrix_data).sum()

0.0

In [221]:
seqs = table1.ids('observation')

In [222]:
meta1 = meta1.reindex(table1.ids('sample'))

In [224]:
if not os.path.exists(main_dir + 'Final'):
    os.mkdir(main_dir + 'Final')

In [225]:
with(open(main_dir + 'Final/dna-fragments.fasta','w')) as f:
    seqs = list(seqs)
    for i in range(0, len(list(seqs))):
        print(">" + seqs[i] + "\n" + seqs[i], file=f)

In [226]:
with biom_open(main_dir + 'Final/feature-table.biom', 'w') as f:  
    table1.to_hdf5(f, "AGP")

In [227]:
meta1.to_csv(main_dir + 'Final/meta_data.csv',sep='\t',index_label='#SampleID')